In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vietnamese-stopwords/vietnamese-stopwords.txt
/kaggle/input/list-product-1/list_products.json


In [2]:
!pip install -U sentence-transformers
!pip install pyvi
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import re
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 69.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.2 MB/s eta 0:00:00


In [3]:
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [5]:
def load_stopwords():
    sw = []
    with open("/kaggle/input/vietnamese-stopwords/vietnamese-stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw

stop_words = load_stopwords()

In [6]:
def preprocess_text(sentence, tokenize, stop_words):
    sentence = str(sentence).lower()
    sentence = re.sub(r"[^\w\s]", " ", sentence)
    sentence = re.sub(r"\s+", " ", sentence)
    sentence = tokenize(sentence)
    filtered_words = [word for word in sentence.split() if word not in stop_words]
    return ' '.join(filtered_words)

In [7]:
df = pd.read_json("/kaggle/input/list-product-1/list_products.json", lines = True)

In [8]:
df["preprocessed_name"] = df["name"].map(lambda sentence: preprocess_text(sentence, tokenize, stop_words))

In [9]:
preprocessed_name = df["preprocessed_name"].values

In [ ]:
name_embeddings = model.encode(preprocessed_name)

Batches:   0%|          | 0/1466 [00:00<?, ?it/s]

In [ ]:
np.savez('product_name_embedding_vector.npz', vectorized_product_names = name_embeddings)